<a href="https://colab.research.google.com/github/azizamirsaidova/fake-news-detection/blob/main/notebooks/fake_news_detection_using_machine_learning_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/azizamirsaidova/fake-news-detection.git

Cloning into 'fake-news-detection'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 80 (delta 24), reused 16 (delta 4), pack-reused 0
Unpacking objects: 100% (80/80), done.


In [ ]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
false = pd.read_csv('/content/false_results.csv')
true = pd.read_csv('/content/true_results.csv')

In [ ]:
false

,text
0,[[{'generated_text': 'Says the Annies List pol...
1,[{'generated_text': 'Health care reform legis...
2,[{'generated_text': 'When Mitt Romney was gov...
3,[{'generated_text': 'Women and men both are m...
4,[{'generated_text': 'I dont know who (Jonatha...
...,...
995,[{'generated_text': 'We have no idea what is ...
996,[{'generated_text': 'Says new estimates from ...
997,[{'generated_text': 'Mitt Romney has 15 homes...
998,[{'generated_text': '[State Rep. Doug McKilli...


In [ ]:
# line = list(df['text'])
#line = line.replace("'generated_text':", '')
false_replaced = map(lambda x: x.replace("'generated_text':",''), false)

In [ ]:
false = false['text'].replace("'generated_text':",'', regex=True).to_frame()
true = true['text'].replace("'generated_text':",'', regex=True).to_frame()
true['label'] = 1
false['label'] = 0

In [ ]:
frames = [true, false]
df = pd.concat(frames)
df

,text,label
0,[{ 'The Chicago Bears have had more starting q...,1
1,"[{ ""McCain opposed a requirement that the gov...",1
2,[{ 'Says Paul Ryan is still endorsing Trump. ...,1
3,"[{ ""We have a federal government that thinks ...",1
4,[{ 'Austin is a city that has basically doubl...,1
...,...,...
995,[{ 'We have no idea what is contained in [ele...,0
996,[{ 'Says new estimates from the Congressional...,0
997,[{ 'Mitt Romney has 15 homes. John McCain has...,0
998,[{ '[State Rep. Doug McKillips] donors deserv...,0


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df['text'], df['label'], test_size=0.2, random_state=8)

In [ ]:
#DataFlair - Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
#DataFlair - Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [ ]:
#DataFlair - Initialize a PassiveAggressiveClassifier
pac=PassiveAggressiveClassifier(max_iter=1000)
pac.fit(tfidf_train,y_train)
#DataFlair - Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 59.25%


## **Other Classification Techniques**

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

<2x2 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [ ]:
x_df = df['text']
y_df = df['label']

In [ ]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(x_df)
freq_term_matrix = count_vectorizer.transform(x_df)
tfidf = TfidfTransformer(norm = "l2")
tfidf.fit(freq_term_matrix)
tf_idf_matrix = tfidf.fit_transform(freq_term_matrix)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(tf_idf_matrix,y_df, random_state=2)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Accuracy = logreg.score(X_test, Y_test)
print(Accuracy*100)

58.199999999999996


In [ ]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()
NB.fit(X_train, Y_train)
Accuracy_NB = NB.score(X_test, Y_test)
print(Accuracy_NB*100)

59.199999999999996


In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)
Accuracy_DT = clf.score(X_test, Y_test)
print(Accuracy_DT*100)

56.39999999999999


## **BERT**

In [ ]:
!pip install "tensorflow>=1.7.0"
!pip install tensorflow-hub

In [ ]:
from sklearn.model_selection import train_test_split  
#Split data into training and testing dataset 
title_train, title_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2,random_state=1000)

In [ ]:
# Use the bert preprocesser and bert encoder from tensorflow_hub 
# !pip install tensorflow_text
import tensorflow_hub as hub
import tensorflow_text
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3") 
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
import tensorflow as tf

# Input Layers
input_layer = tf.keras.layers.Input(shape=(), dtype=tf.string, name='news')

# BERT layers
processed = bert_preprocess(input_layer)
output = bert_encoder(processed)

# # Fully Connected Layers
layer = tf.keras.layers.Dropout(0.2, name='dropout')(output['pooled_output'])
layer = tf.keras.layers.Dense(10,activation='relu', name='hidden')(layer)
layer = tf.keras.layers.Dense(1,activation='sigmoid', name='output')(layer)


model = tf.keras.Model(inputs=[input_layer],outputs=[layer])

#Compile model on adam optimizer, binary_crossentropy loss, and accuracy metrics
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
#Train model on 5 epochs
model.fit(title_train, y_train, epochs= 5)

#Evaluate model on test data
model.evaluate(title_test,y_test)

Epoch 1/5
50/50 [==============================] - 757s 15s/step - loss: 0.6977 - accuracy: 0.4919
Epoch 2/5
50/50 [==============================] - 740s 15s/step - loss: 0.6940 - accuracy: 0.5019
Epoch 3/5
50/50 [==============================] - 733s 15s/step - loss: 0.6937 - accuracy: 0.4931
Epoch 4/5
50/50 [==============================] - 736s 15s/step - loss: 0.6933 - accuracy: 0.4988
Epoch 5/5
13/13 [==============================] - 186s 14s/step - loss: 0.6913 - accuracy: 0.4900


[0.6913196444511414, 0.49000000953674316]